In [1]:
!pip install random-user-agent --root-user-action=ignore --quiet

import foxess as f
import private as my

f.username = my.fox_username
f.password = my.fox_password
f.device_sn = my.fox_device_sn

FoxESS-Cloud version 0.3.7


In [2]:
f.get_info()

{'user': 'tonymatthews89',
 'name': 'Tony Matthews',
 'phone': '07748987537',
 'email': 'tony.matthews1@outlook.com',
 'country': 'GB',
 'city': 'Swindon',
 'address': '89 Okebourne Park',
 'postcode': '',
 'note': '',
 'clientID': '08fd6e5aafa84i4301rae51g6dfd7a779edc',
 'countryName': 'United Kingdom',
 'access': 1}

In [3]:
f.get_site()

{'name': '89 Okebourne Park',
 'capacity': 393216,
 'capacitySolaryaan': '',
 'type': 2,
 'quantity': 1,
 'country': 'United Kingdom',
 'countryCode': 'GB',
 'city': 'Swindon',
 'address': '89 Okebourne Park',
 'postcode': 'SN3 6AJ',
 'price': '',
 'createdDate': '2023-01-10 13:23:49 GMT+0000',
 'status': 1,
 'stationID': 'ffca8cc5-6d3b-4d43-ba55-72d9e0828f0b',
 'systemCapacity': 6400,
 'generationToday': 19.5,
 'power': 2.078,
 'allowEdit': False,
 'position': {'format': 'dd', 'x': '', 'y': '', 'pid': ''},
 'pileSN': '',
 'pileState': 0}

In [4]:
f.get_logger()

{'moduleID': '85044387-f2a4-4bd1-9ff1-42dbe066fce1',
 'moduleSN': '669L3EPF22WA061',
 'moduleType': 'L3',
 'plantName': '89 Okebourne Park',
 'stationID': '',
 'version': '3.08',
 'signal': 0,
 'communication': 1}

In [5]:
f.get_device()

{'deviceID': 'dcd0712d-3434-40db-9323-933288bc5964',
 'deviceSN': '65BH602026EB004',
 'moduleSN': '669L3EPF22WA061',
 'plantName': '89 Okebourne Park',
 'deviceType': 'H1-6.0-E',
 'country': 'United Kingdom',
 'countryCode': '',
 'feedinDate': '2023-01-10 15:09:35 GMT+0000',
 'status': 1,
 'power': 6.0,
 'generationToday': 19.5,
 'generationTotal': 4309.5,
 'productType': 'H',
 'flowType': 2,
 'hasBattery': True,
 'hasPV': True,
 'dataLatestUploadDate': '2023-09-05 16:19:30 BST+0100',
 'hasWifiMeter': False,
 'inParallel': 0,
 'wifiMeterID': '',
 'wifiMeterSN': '',
 'atFlag': False,
 'eps': True,
 'model': 'H1',
 'phase': 1}

In [6]:
f.get_firmware()

{'master': '1.54', 'slave': '1.02', 'manager': '1.57', 'afci': ''}

In [7]:
#f.get_work_mode()
#f.set_work_mode('SelfUse')
f.get_work_mode()

'Feedin'

In [8]:
f.get_battery()

{'status': 1,
 'soc': 99,
 'volt': 319.6,
 'current': 0,
 'power': 0.054,
 'residual': 13480,
 'temperature': 33.3,
 'timestamp': '',
 'warning': False,
 'descUrl': ''}

In [9]:
f.get_settings()

{'times': [{'tip': '',
   'enableCharge': True,
   'enableGrid': False,
   'startTime': {'hour': 0, 'minute': 0},
   'endTime': {'hour': 0, 'minute': 0}},
  {'tip': '',
   'enableCharge': True,
   'enableGrid': False,
   'startTime': {'hour': 0, 'minute': 0},
   'endTime': {'hour': 0, 'minute': 0}}],
 'minSoc': 10,
 'minGridSoc': 12}

In [10]:
f.get_earnings()

{'currency': 'GBP(￡)',
 'power': 2.078,
 'today': {'generation': 19.5, 'earnings': 4.095},
 'month': {'generation': 97.89999999999964, 'earnings': 20.558999999999923},
 'year': {'generation': 4309.5, 'earnings': 904.995},
 'cumulate': {'generation': 4309.5, 'earnings': 904.995},
 'systemCapacity': 393216}

In [11]:
f.raw_vars[:10]

[{'name': 'PV1Volt', 'variable': 'pv1Volt', 'unit': 'V'},
 {'name': 'PV1Current', 'variable': 'pv1Current', 'unit': 'A'},
 {'name': 'PV1Power', 'variable': 'pv1Power', 'unit': 'kW'},
 {'name': 'PVPower', 'variable': 'pvPower', 'unit': 'kW'},
 {'name': 'PV2Volt', 'variable': 'pv2Volt', 'unit': 'V'},
 {'name': 'PV2Current', 'variable': 'pv2Current', 'unit': 'A'},
 {'name': 'PV2Power', 'variable': 'pv2Power', 'unit': 'kW'},
 {'name': 'PV3Volt', 'variable': 'pv3Volt', 'unit': 'V'},
 {'name': 'PV3Current', 'variable': 'pv3Current', 'unit': 'A'},
 {'name': 'PV3Power', 'variable': 'pv3Power', 'unit': 'kW'}]

In [12]:
f.get_raw(v='generationPower', summary=1)

[{'variable': 'generationPower',
  'unit': 'kW',
  'name': 'Output Power',
  'data': [{'time': '2023-09-05 16:04:29 BST+0100', 'value': 2.164},
   {'time': '2023-09-05 16:09:30 BST+0100', 'value': 2.114},
   {'time': '2023-09-05 16:14:30 BST+0100', 'value': 2.105},
   {'time': '2023-09-05 16:19:30 BST+0100', 'value': 2.078}]}]

In [13]:
f.get_raw('week', d='2023-08-21',v=['RVolt'], summary=2)

[{'variable': 'RVolt',
  'unit': 'V',
  'name': 'RVolt',
  'date': '2023-08-15',
  'count': 288,
  'average': 240.115,
  'max': 248.1,
  'max_time': '16:13',
  'min': 235.1,
  'min_time': '12:08'},
 {'variable': 'RVolt',
  'unit': 'V',
  'name': 'RVolt',
  'date': '2023-08-16',
  'count': 288,
  'average': 241.121,
  'max': 249.6,
  'max_time': '12:33',
  'min': 234.9,
  'min_time': '20:38'},
 {'variable': 'RVolt',
  'unit': 'V',
  'name': 'RVolt',
  'date': '2023-08-17',
  'count': 288,
  'average': 239.999,
  'max': 250.2,
  'max_time': '14:39',
  'min': 233.6,
  'min_time': '19:59'},
 {'variable': 'RVolt',
  'unit': 'V',
  'name': 'RVolt',
  'date': '2023-08-18',
  'count': 288,
  'average': 238.131,
  'max': 242.5,
  'max_time': '17:19',
  'min': 231.6,
  'min_time': '12:59'},
 {'variable': 'RVolt',
  'unit': 'V',
  'name': 'RVolt',
  'date': '2023-08-19',
  'count': 288,
  'average': 239.047,
  'max': 248.9,
  'max_time': '15:24',
  'min': 234,
  'min_time': '07:29'},
 {'variable'

In [14]:
f.get_raw('week', d='2023-08-21',v=['pvPower'], summary=2)

[{'variable': 'pvPower',
  'unit': 'kWh',
  'name': 'pv_energy_daily',
  'kwh': 23.705,
  'kwh_off': 0.0,
  'kwh_peak': 5.243,
  'kwh_neg': 0.0,
  'date': '2023-08-15',
  'count': 288,
  'average': 0.988,
  'max': 5.669,
  'max_time': '12:18',
  'min': 0,
  'min_time': '00:03'},
 {'variable': 'pvPower',
  'unit': 'kWh',
  'name': 'pv_energy_daily',
  'kwh': 30.598,
  'kwh_off': 0.0,
  'kwh_peak': 5.031,
  'kwh_neg': 0.0,
  'date': '2023-08-16',
  'count': 288,
  'average': 1.275,
  'max': 4.692,
  'max_time': '14:08',
  'min': 0,
  'min_time': '00:03'},
 {'variable': 'pvPower',
  'unit': 'kWh',
  'name': 'pv_energy_daily',
  'kwh': 23.778,
  'kwh_off': 0.0,
  'kwh_peak': 2.063,
  'kwh_neg': 0.0,
  'date': '2023-08-17',
  'count': 288,
  'average': 0.991,
  'max': 4.959,
  'max_time': '13:24',
  'min': 0,
  'min_time': '00:04'},
 {'variable': 'pvPower',
  'unit': 'kWh',
  'name': 'pv_energy_daily',
  'kwh': 8.404,
  'kwh_off': 0.0,
  'kwh_peak': 1.37,
  'kwh_neg': 0.0,
  'date': '2023-0

In [15]:
f.get_raw('day', d='2023-09-01',v=['RPower'], summary=2, save='rpower')

[{'variable': 'RPower',
  'unit': 'kWh',
  'name': 'RPower',
  'kwh': 12.355,
  'kwh_off': 0.561,
  'kwh_peak': 3.666,
  'kwh_neg': 4.418,
  'date': '2023-09-01',
  'count': 287,
  'average': 0.332,
  'max': 3.35,
  'max_time': '18:53',
  'min': -4.446,
  'min_time': '02:58'}]

In [16]:
f.get_raw('day', d='2023-09-01',v=['RPower'], summary=2, load='rpower_raw_day_20230901.json')

[{'variable': 'RPower',
  'unit': 'kWh',
  'name': 'RPower',
  'kwh': 12.355,
  'kwh_off': 0.561,
  'kwh_peak': 3.666,
  'kwh_neg': 4.418,
  'date': '2023-09-01',
  'count': 287,
  'average': 0.332,
  'max': 3.35,
  'max_time': '18:53',
  'min': -4.446,
  'min_time': '02:58'}]

In [17]:
f.flip_ct2 = True
f.get_raw('day', d='2023-08-21',v=['pvPower','meterPower2'], summary=2)

[{'variable': 'pvPower',
  'unit': 'kWh',
  'name': 'pv_energy_daily',
  'kwh': 22.205,
  'kwh_off': 0.0,
  'kwh_peak': 4.941,
  'kwh_neg': 0.0,
  'date': '2023-08-21',
  'count': 288,
  'average': 0.925,
  'max': 5.366,
  'max_time': '12:50',
  'min': 0,
  'min_time': '00:00'},
 {'variable': 'meterPower2',
  'unit': 'kWh',
  'name': 'ct2_daily',
  'kwh': 0.0,
  'kwh_off': 0.0,
  'kwh_peak': 0.0,
  'kwh_neg': 0.0,
  'date': '2023-08-21',
  'count': 288,
  'average': 0.0,
  'max': 0,
  'max_time': '00:00',
  'min': 0,
  'min_time': '00:00'}]

In [18]:
d = '2023-05-17 00:00:00'
result=f.get_raw('day', d=d, v=f.power_vars, summary=3)
for r in result:
    print(f"{r['name']:20} = {round(r['kwh'],3)}")

output_daily         = 19.663
feedin_daily         = 8.523
load_daily           = 11.984
grid_daily           = 0.864
bat_charge_daily     = 6.725
bat_discharge_daily  = 5.248
pv_energy_daily      = 23.554
ct2_daily            = 0.0
input_daily          = 0.0


In [19]:
result=f.get_report('month', d=d, v=f.report_vars)
i = int(d[8:10])-1
for r in result:
    print(f"{r['variable']:20} = {round(r['data'][i]['value'],3)}")

generation           = 20
feedin               = 8
loads                = 12.5
gridConsumption      = 0.7
chargeEnergyToTal    = 6.9
dischargeEnergyToTal = 5.5


In [20]:
f.get_report('day', d=d, v='feedin', summary=2)

[{'variable': 'feedin', 'data': [], 'total': 8}]

In [21]:
f.get_report('week', d=d, v='feedin', summary=2, save='feedin')

[{'variable': 'feedin',
  'data': [{'index': 11, 'value': 4.800000000000011},
   {'index': 12, 'value': 1},
   {'index': 13, 'value': 1.5},
   {'index': 14, 'value': 12.5},
   {'index': 15, 'value': 12.5},
   {'index': 16, 'value': 13.199999999999989},
   {'index': 17, 'value': 8}],
  'total': 53.5,
  'sum': 53.5,
  'average': 7.643,
  'date': '2023-05-17 00:00:00',
  'count': 7,
  'max': 13.2,
  'max_index': 5,
  'min': 1,
  'min_index': 1}]

In [22]:
f.get_report('week', v='loads')

[{'variable': 'loads',
  'data': [{'index': 29, 'value': 13.199999999999818},
   {'index': 30, 'value': 15.700000000000273},
   {'index': 31, 'value': 15.199999999999818},
   {'index': 1, 'value': 12.099999999999909},
   {'index': 2, 'value': 13.600000000000364},
   {'index': 3, 'value': 9.299999999999727},
   {'index': 4, 'value': 10.5}],
  'total': 89.6,
  'sum': 89.6,
  'average': 12.8,
  'date': '2023-09-04',
  'count': 7,
  'max': 15.7,
  'max_index': 1,
  'min': 9.3,
  'min_index': 5}]

In [23]:
f.get_report('year', d=d, v='generation')

[{'variable': 'generation',
  'data': [{'index': 1, 'value': 87.89999999999999},
   {'index': 2, 'value': 246.3},
   {'index': 3, 'value': 464.0000000000002},
   {'index': 4, 'value': 625.0000000000003},
   {'index': 5, 'value': 740.9999999999995},
   {'index': 6, 'value': 773.0999999999999},
   {'index': 7, 'value': 618.2000000000003},
   {'index': 8, 'value': 619.5999999999999},
   {'index': 9, 'value': 97.69999999999891}],
  'total': 4272.8,
  'sum': 4272.8,
  'average': 474.756,
  'date': '2023-05-17 00:00:00',
  'count': 9,
  'max': 773.1,
  'max_index': 5,
  'min': 87.9,
  'min_index': 0}]